## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [2]:
healty_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/healty-dataset"
diseased_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Raw_Recordings"

In [3]:
as_ = np.load(diseased_path + "/trimmed_5_as.npy", allow_pickle= True)

In [4]:
as_.shape

(3, 139, 2560)

In [5]:
healty_ = np.load(healty_path + "/trimmed_5_healty.npy", allow_pickle = True)

In [6]:
def converter(array):
    x,y,z = [],[],[]
    for i in range(array.shape[1]):
        x.append(obspy.core.trace.Trace(array[0][i]))
        y.append(obspy.core.trace.Trace(array[1][i]))
        z.append(obspy.core.trace.Trace(array[2][i]))
        
        x[i].stats.sampling_rate = 256
        y[i].stats.sampling_rate = 256
        z[i].stats.sampling_rate = 256
    return [x,y,z]

In [7]:
as_ = converter(as_)
healty_ = converter(healty_)

In [8]:
#sanity check
for i in range(len(as_[1])):
    print(as_[0][0] == as_[2][i])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [9]:
from obspy.imaging.cm import obspy_sequential
import math
from matplotlib import mlab
from matplotlib.colors import Normalize

In [10]:
def _nearest_pow_2(x):
    a = math.pow(2, math.ceil(np.log2(x)))
    b = math.pow(2, math.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [11]:
def output_spectrogram(data, samp_rate, per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=obspy_sequential, zorder=None, title=None,
                show=True, clip=[0.0, 1.0]):
 
    import matplotlib.pyplot as plt
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - np.array(data).mean()
    end = npts / samp_rate

    # Here we call not plt.specgram as this already produces a plot
    # matplotlib.mlab.specgram should be faster as it computes only the
    # arrays
    # XXX mlab.specgram uses fft, would be better and faster use rfft
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap, mode = 'psd')

    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]
    
    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)

    # calculate half bin width
    halfbin_time = (time[1] - time[0]) / 2.0
    halfbin_freq = (freq[1] - freq[0]) / 2.0

    # argument None is not allowed for kwargs on matplotlib python 3.3
    kwargs = {k: v for k, v in (('cmap', cmap), ('zorder', zorder))
              if v is not None}

    if log:
        # pcolor expects one bin more at the right end
        freq = np.concatenate((freq, [freq[-1] + 2 * halfbin_freq]))
        time = np.concatenate((time, [time[-1] + 2 * halfbin_time]))
        # center bin
        time -= halfbin_time
        freq -= halfbin_freq
        # Log scaling for frequency values (y-axis)
        ax.set_yscale('log')
        # Plot times
        ax.pcolormesh(time, freq, specgram, norm=norm, **kwargs)
    else:
        # this method is much much faster!
        specgram = np.flipud(specgram)
        # center bin
        extent = (time[0] - halfbin_time, time[-1] + halfbin_time,
                  freq[0] - halfbin_freq, freq[-1] + halfbin_freq)
    return [specgram, extent, end]

    



In [12]:
def freq_average(array):
    means= []
    for i in range(array.shape[0]):
        means.append(array[i][0:40].mean())
    return np.array(means)
        

In [13]:
def spec_values(array):
    x_, y_, z_ = [],[],[]
    for i in range(len(array[0])):
        _x = output_spectrogram(array[0][i].data, 256)[0]
        _y = output_spectrogram(array[1][i].data, 256)[0]
        _z = output_spectrogram(array[2][i].data, 256)[0]

        x_.append(freq_average(array = _x))
        y_.append(freq_average(array = _y))
        z_.append(freq_average(array = _z)) 
        
        
    return np.stack((x_, y_, z_), axis = 0)

In [14]:
healty = spec_values(healty_)
ass = spec_values(as_)

In [15]:
print(ass.shape)
print(healty.shape)


(3, 139, 2048)
(3, 282, 2048)


In [16]:
ass[0].shape

(139, 2048)

In [17]:
ass[0][70]

array([0.00057072, 0.00081651, 0.00082678, ..., 0.03975494, 0.04164825,
       0.04280185])

# apply mann whitney u test 

In [18]:
from scipy.stats import mannwhitneyu

In [19]:
from scipy import stats

###  x components

In [20]:
test = mannwhitneyu(healty[0], ass[0], use_continuity=False, alternative = 'two-sided')
df_stats_x = pd.DataFrame(test[0])
df_p_x = pd.DataFrame(test[1])
df_p_x

,0
0,1.471893e-62
1,1.471893e-62
2,1.471893e-62
3,1.471893e-62
4,1.471893e-62
...,...
2043,1.471893e-62
2044,1.471893e-62
2045,1.471893e-62
2046,1.471893e-62


In [21]:
df_p_x.style.apply(lambda x: ["background: yellow" if v < 0.05 else "" for v in x], axis = 1)

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000


### -------- with two sample t test

In [22]:
p_val = pd.DataFrame(stats.ttest_ind(healty[0], ass[0])[1])
p_val.style.apply(lambda x: ["background: pink" if v < 0.05 else "" for v in x], axis = 1)

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000


### y component

In [23]:
test_y = mannwhitneyu(healty[1], ass[1], use_continuity=False, alternative = 'two-sided')
df_stats_y = pd.DataFrame(test_y[1])
df_p_y = pd.DataFrame(test_y[1])
df_p_y

,0
0,1.471893e-62
1,1.471893e-62
2,1.471893e-62
3,1.471893e-62
4,1.471893e-62
...,...
2043,1.471893e-62
2044,1.471893e-62
2045,1.471893e-62
2046,1.471893e-62


In [24]:
df_p_y.style.apply(lambda x: ["background: yellow" if v < 0.05 else "" for v in x], axis = 1)

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000


### --------- with two sample t test

In [25]:
p_val_y = pd.DataFrame(stats.ttest_ind(healty[1], ass[1])[1])
p_val_y.style.apply(lambda x: ["background: pink" if v < 0.05 else "" for v in x], axis = 1)

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000


### z component

In [26]:
test_z = mannwhitneyu(healty[2], ass[2], use_continuity=False, alternative = 'two-sided')
df_stats_z = pd.DataFrame(test_z[1])
df_p_z = pd.DataFrame(test_z[1])
df_p_z

,0
0,1.471893e-62
1,1.471893e-62
2,1.471893e-62
3,1.471893e-62
4,1.471893e-62
...,...
2043,1.471893e-62
2044,1.471893e-62
2045,1.471893e-62
2046,1.471893e-62


In [27]:
df_p_z.style.apply(lambda x: ["background: yellow" if v < 0.05 else "" for v in x], axis = 1)

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000


### --------- with two sample t test

In [28]:
p_val_y = pd.DataFrame(stats.ttest_ind(healty[1], ass[1])[1])
p_val_y.style.apply(lambda x: ["background: pink" if v < 0.05 else "" for v in x], axis = 1)

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
